# EE 451: Communications Systems
## Lecture 13 - BPSK Theory & Implementation

### Learning Objectives

By the end of this notebook, you will be able to:

1. Explain Binary Phase Shift Keying (BPSK) modulation principles
2. Analyze BPSK constellation diagrams and signal space representations
3. Calculate and interpret BPSK power spectral density
4. Compare BPSK spectral efficiency to ASK and FSK
5. Generate BPSK signals in Python simulations
6. Design BPSK modulators using product modulators

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq, fftshift

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## Part 1: BPSK Fundamentals

Binary Phase Shift Keying (BPSK) is a digital modulation technique where:
- Bit "1": $s_1(t) = A \cos(2\pi f_c t)$
- Bit "0": $s_0(t) = A \cos(2\pi f_c t + \pi) = -A \cos(2\pi f_c t)$

The two symbols differ by a 180° phase shift, making BPSK an **antipodal signaling** scheme.

### Mathematical Representation
$$s(t) = A \cdot b(t) \cdot \cos(2\pi f_c t)$$

where $b(t) = +1$ for bit "1" and $b(t) = -1$ for bit "0".

In [ ]:
# BPSK Parameters
R_b = 1000  # Bit rate (bps)
T_b = 1 / R_b  # Bit duration
f_c = 10000  # Carrier frequency (Hz)
A = 1  # Amplitude
fs = 100000  # Sampling rate (Hz)
num_bits = 10

# Generate random bits
np.random.seed(42)  # For reproducibility
bits = np.random.randint(0, 2, num_bits)
print(f"Transmitted bits: {bits}")

# NRZ encoding: 0 → -1, 1 → +1
nrz = 2 * bits - 1  # Maps 0→-1, 1→+1

# Generate time vector
t = np.arange(0, num_bits * T_b, 1/fs)

# Create baseband signal (rectangular pulses)
baseband = np.zeros(len(t))
for n in range(num_bits):
    start_idx = int(n * T_b * fs)
    end_idx = int((n + 1) * T_b * fs)
    if end_idx <= len(baseband):
        baseband[start_idx:end_idx] = nrz[n]

# Generate carrier
carrier = A * np.cos(2 * np.pi * f_c * t[:len(baseband)])

# BPSK modulation: multiply baseband by carrier
s_bpsk = baseband * carrier

# Plot results
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Binary data
bit_times = np.arange(num_bits)
axes[0].step(bit_times, bits, where='post', linewidth=2, color='blue')
axes[0].set_ylabel('Bits')
axes[0].set_title('Binary Data')
axes[0].set_ylim(-0.5, 1.5)
axes[0].set_yticks([0, 1])
axes[0].grid(True, alpha=0.3)

# NRZ baseband
axes[1].plot(t * 1000, baseband, linewidth=2, color='green')
axes[1].set_ylabel('Amplitude')
axes[1].set_title('NRZ Baseband Signal (±1)')
axes[1].set_ylim(-1.5, 1.5)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linewidth=0.5)

# Carrier
plot_samples = int(5 * T_b * fs)  # Show first 5 bits only for clarity
axes[2].plot(t[:plot_samples] * 1000, carrier[:plot_samples], linewidth=1, color='red')
axes[2].set_ylabel('Amplitude')
axes[2].set_title(f'Carrier Signal ({f_c} Hz)')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='k', linewidth=0.5)

# BPSK signal
axes[3].plot(t[:plot_samples] * 1000, s_bpsk[:plot_samples], linewidth=1, color='purple')
axes[3].plot(t[:plot_samples] * 1000, baseband[:plot_samples], '--', linewidth=2, color='green', alpha=0.5, label='Envelope')
axes[3].plot(t[:plot_samples] * 1000, -baseband[:plot_samples], '--', linewidth=2, color='green', alpha=0.5)
axes[3].set_xlabel('Time (ms)')
axes[3].set_ylabel('Amplitude')
axes[3].set_title('BPSK Modulated Signal')
axes[3].legend()
axes[3].grid(True, alpha=0.3)
axes[3].axhline(y=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

print(f"\nBPSK Parameters:")
print(f"Bit rate: {R_b} bps")
print(f"Carrier frequency: {f_c} Hz")
print(f"Amplitude: {A} V")
print(f"\nNote: Phase flips 180° when bits change from 1→0 or 0→1")

## Part 2: BPSK Constellation Diagram

The constellation diagram represents signal symbols in the complex I-Q plane:
- **I (In-phase)**: Real component
- **Q (Quadrature)**: Imaginary component

For BPSK:
- Bit "1": Symbol at $(+A, 0)$
- Bit "0": Symbol at $(-A, 0)$

BPSK is a **one-dimensional** modulation (I-axis only, Q = 0).

### Signal Energy
Energy per bit: $E_b = \frac{A^2 T_b}{2}$

In [ ]:
# BPSK constellation points
I_symbols = np.array([A, -A])  # In-phase component
Q_symbols = np.array([0, 0])   # Quadrature component (always 0 for BPSK)
labels = ['Bit 1', 'Bit 0']
colors = ['blue', 'red']

# Calculate energy per bit
E_b = (A**2 * T_b) / 2
distance = 2 * A  # Euclidean distance between symbols

# Plot constellation
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Constellation diagram
for i in range(2):
    axes[0].plot(I_symbols[i], Q_symbols[i], 'o', markersize=20, color=colors[i], label=labels[i])
    axes[0].annotate(labels[i], (I_symbols[i], Q_symbols[i]), 
                    textcoords="offset points", xytext=(0,15), ha='center', fontsize=12)

# Decision boundary
axes[0].axvline(x=0, color='green', linestyle='--', linewidth=2, label='Decision Boundary')

# Formatting
axes[0].set_xlabel('In-phase (I)', fontsize=12)
axes[0].set_ylabel('Quadrature (Q)', fontsize=12)
axes[0].set_title('BPSK Constellation Diagram', fontsize=14)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linewidth=0.5)
axes[0].set_xlim(-1.5*A, 1.5*A)
axes[0].set_ylim(-0.5*A, 0.5*A)
axes[0].legend(fontsize=10)
axes[0].set_aspect('equal')

# Signal space representation
axes[1].plot([0, I_symbols[0]], [0, 0], 'o-', linewidth=3, markersize=15, color='blue', label='s₁(t) - Bit 1')
axes[1].plot([0, I_symbols[1]], [0, 0], 'o-', linewidth=3, markersize=15, color='red', label='s₀(t) - Bit 0')
axes[1].annotate(f'Distance = {distance:.1f}', 
                xy=(0, -0.15), ha='center', fontsize=12, 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

axes[1].set_xlabel('Signal Space Dimension φ₁(t)', fontsize=12)
axes[1].set_ylabel('Amplitude', fontsize=12)
axes[1].set_title('BPSK Signal Space (Antipodal)', fontsize=14)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linewidth=0.5)
axes[1].axvline(x=0, color='k', linewidth=0.5)
axes[1].set_xlim(-1.5*A, 1.5*A)
axes[1].set_ylim(-0.3*A, 0.3*A)
axes[1].legend(fontsize=10)

plt.tight_layout()
plt.show()

print(f"\nBPSK Constellation Properties:")
print(f"Symbol 1 (Bit '1'): I = +{A}, Q = 0")
print(f"Symbol 0 (Bit '0'): I = -{A}, Q = 0")
print(f"\nEnergy per bit: E_b = {E_b:.6f} J")
print(f"Euclidean distance: d = {distance} (maximum for given energy)")
print(f"Decision rule: If I > 0 → Bit '1', else → Bit '0'")

## Part 3: BPSK Power Spectral Density

The power spectral density (PSD) of BPSK is determined by:
1. **Baseband PSD**: Binary NRZ pulses → $S_{bb}(f) \propto T_b \text{sinc}^2(\pi f T_b)$
2. **Modulation**: Shifts baseband to $\pm f_c$

$$S_{BPSK}(f) \propto \text{sinc}^2[\pi(f - f_c)T_b] + \text{sinc}^2[\pi(f + f_c)T_b]$$

### Bandwidth
- **Null-to-null**: $BW = 2R_b = 2/T_b$
- **99% power**: $BW \approx 1.2 R_b$

In [ ]:
# Generate longer BPSK signal for better spectral estimate
num_bits_psd = 1000
bits_psd = np.random.randint(0, 2, num_bits_psd)
nrz_psd = 2 * bits_psd - 1

t_psd = np.arange(0, num_bits_psd * T_b, 1/fs)
baseband_psd = np.zeros(len(t_psd))

for n in range(num_bits_psd):
    start_idx = int(n * T_b * fs)
    end_idx = int((n + 1) * T_b * fs)
    if end_idx <= len(baseband_psd):
        baseband_psd[start_idx:end_idx] = nrz_psd[n]

carrier_psd = np.cos(2 * np.pi * f_c * t_psd[:len(baseband_psd)])
s_bpsk_psd = baseband_psd * carrier_psd

# Compute power spectral density using FFT
S = fftshift(fft(s_bpsk_psd))
f = fftshift(fftfreq(len(s_bpsk_psd), 1/fs))
PSD = (np.abs(S)**2) / len(s_bpsk_psd)
PSD_dB = 10 * np.log10(PSD + 1e-12)  # Add small value to avoid log(0)

# Baseband PSD
S_bb = fftshift(fft(baseband_psd))
PSD_bb = (np.abs(S_bb)**2) / len(baseband_psd)
PSD_bb_dB = 10 * np.log10(PSD_bb + 1e-12)

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Baseband spectrum
axes[0].plot(f / 1000, PSD_bb_dB, linewidth=1.5, color='green')
axes[0].axvline(x=R_b/1000, color='red', linestyle='--', linewidth=2, label=f'R_b = {R_b} Hz')
axes[0].axvline(x=-R_b/1000, color='red', linestyle='--', linewidth=2)
axes[0].axvline(x=2*R_b/1000, color='orange', linestyle='--', linewidth=2, label=f'2R_b = {2*R_b} Hz')
axes[0].axvline(x=-2*R_b/1000, color='orange', linestyle='--', linewidth=2)
axes[0].set_xlabel('Frequency (kHz)')
axes[0].set_ylabel('PSD (dB)')
axes[0].set_title('Baseband NRZ Spectrum')
axes[0].set_xlim(-5, 5)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# BPSK spectrum
axes[1].plot(f / 1000, PSD_dB, linewidth=1.5, color='purple')
axes[1].axvline(x=f_c/1000, color='blue', linestyle='--', linewidth=2, label=f'Carrier = {f_c} Hz')
axes[1].axvline(x=-f_c/1000, color='blue', linestyle='--', linewidth=2)
axes[1].axvline(x=(f_c + R_b)/1000, color='red', linestyle='--', linewidth=1, alpha=0.7, label='First null')
axes[1].axvline(x=(f_c - R_b)/1000, color='red', linestyle='--', linewidth=1, alpha=0.7)
axes[1].axvline(x=-(f_c + R_b)/1000, color='red', linestyle='--', linewidth=1, alpha=0.7)
axes[1].axvline(x=-(f_c - R_b)/1000, color='red', linestyle='--', linewidth=1, alpha=0.7)
axes[1].set_xlabel('Frequency (kHz)')
axes[1].set_ylabel('PSD (dB)')
axes[1].set_title(f'BPSK Spectrum (R_b = {R_b} bps, Null-to-Null BW = {2*R_b} Hz)')
axes[1].set_xlim(-15, 15)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBPSK Spectral Properties:")
print(f"Bit rate: R_b = {R_b} bps")
print(f"Null-to-null bandwidth: {2*R_b} Hz")
print(f"99% power bandwidth: ~{1.2*R_b:.0f} Hz")
print(f"Spectral efficiency: {R_b/(2*R_b)} bits/s/Hz = 0.5 bits/s/Hz")

## Part 4: Comparison with ASK and FSK

Let's compare BPSK with other binary modulation schemes:

| Modulation | Null-to-Null BW | 99% Power BW | Spectral Efficiency | Noise Performance |
|------------|-----------------|--------------|---------------------|-------------------|
| ASK (OOK)  | 2R_b           | ~R_b         | 0.5 bits/s/Hz       | Poor (amplitude) |
| FSK (h=1)  | 3R_b           | ~2.5R_b      | 0.33 bits/s/Hz      | Moderate |
| BPSK       | 2R_b           | ~1.2R_b      | 0.5 bits/s/Hz       | Best (phase) |

### Key Advantages of BPSK:
1. **Constant envelope**: Power efficient, works with nonlinear amplifiers
2. **Best noise immunity**: Maximum Euclidean distance for binary signaling
3. **Same bandwidth as ASK**: But better SNR performance (3 dB advantage)
4. **More bandwidth efficient than FSK**: 33% improvement

In [ ]:
# Generate comparison signals
num_bits_comp = 100
bits_comp = np.random.randint(0, 2, num_bits_comp)
nrz_comp = 2 * bits_comp - 1

t_comp = np.arange(0, num_bits_comp * T_b, 1/fs)
baseband_comp = np.zeros(len(t_comp))

for n in range(num_bits_comp):
    start_idx = int(n * T_b * fs)
    end_idx = int((n + 1) * T_b * fs)
    if end_idx <= len(baseband_comp):
        baseband_comp[start_idx:end_idx] = nrz_comp[n]

# ASK (OOK): 0 → 0, 1 → A
ask_baseband = (bits_comp.repeat(int(T_b * fs)))[:len(t_comp)]
s_ask = ask_baseband * np.cos(2 * np.pi * f_c * t_comp)

# BPSK
s_bpsk_comp = baseband_comp * np.cos(2 * np.pi * f_c * t_comp)

# FSK (simplified - frequency shift)
f_mark = f_c + R_b / 2  # Frequency for bit 1
f_space = f_c - R_b / 2  # Frequency for bit 0
s_fsk = np.zeros(len(t_comp))
for n in range(num_bits_comp):
    start_idx = int(n * T_b * fs)
    end_idx = int((n + 1) * T_b * fs)
    if end_idx <= len(s_fsk):
        if bits_comp[n] == 1:
            s_fsk[start_idx:end_idx] = np.cos(2 * np.pi * f_mark * t_comp[start_idx:end_idx])
        else:
            s_fsk[start_idx:end_idx] = np.cos(2 * np.pi * f_space * t_comp[start_idx:end_idx])

# Compute spectra
S_ask = fftshift(fft(s_ask))
S_bpsk_comp = fftshift(fft(s_bpsk_comp))
S_fsk = fftshift(fft(s_fsk))
f_spec = fftshift(fftfreq(len(t_comp), 1/fs))

PSD_ask = (np.abs(S_ask)**2) / len(s_ask)
PSD_bpsk_comp = (np.abs(S_bpsk_comp)**2) / len(s_bpsk_comp)
PSD_fsk = (np.abs(S_fsk)**2) / len(s_fsk)

# Normalize for comparison
PSD_ask_norm = PSD_ask / np.max(PSD_ask)
PSD_bpsk_norm = PSD_bpsk_comp / np.max(PSD_bpsk_comp)
PSD_fsk_norm = PSD_fsk / np.max(PSD_fsk)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Time domain (first 10 bits)
plot_samples = int(10 * T_b * fs)
t_plot = t_comp[:plot_samples] * 1000

axes[0].plot(t_plot, s_ask[:plot_samples], linewidth=1, label='ASK (OOK)', alpha=0.7)
axes[0].plot(t_plot, s_bpsk_comp[:plot_samples], linewidth=1, label='BPSK', alpha=0.7)
axes[0].plot(t_plot, s_fsk[:plot_samples], linewidth=1, label='FSK', alpha=0.7)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Modulation Comparison - Time Domain')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Frequency domain
axes[1].plot(f_spec / 1000, 10*np.log10(PSD_ask_norm + 1e-10), linewidth=2, label='ASK (OOK)', alpha=0.7)
axes[1].plot(f_spec / 1000, 10*np.log10(PSD_bpsk_norm + 1e-10), linewidth=2, label='BPSK', alpha=0.7)
axes[1].plot(f_spec / 1000, 10*np.log10(PSD_fsk_norm + 1e-10), linewidth=2, label='FSK', alpha=0.7)
axes[1].axvline(x=f_c/1000, color='k', linestyle='--', linewidth=1, alpha=0.5, label='Carrier')
axes[1].set_xlabel('Frequency (kHz)')
axes[1].set_ylabel('Normalized PSD (dB)')
axes[1].set_title('Modulation Comparison - Frequency Domain')
axes[1].set_xlim(8, 12)
axes[1].set_ylim(-40, 5)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nModulation Comparison Summary:")
print("═" * 70)
print(f"{'Modulation':<15} {'Null-Null BW':<15} {'Efficiency':<20} {'Notes'}")
print("─" * 70)
print(f"{'ASK (OOK)':<15} {f'{2*R_b} Hz':<15} {'0.5 bits/s/Hz':<20} {'Amplitude varies'}")
print(f"{'FSK (h=1)':<15} {f'{3*R_b} Hz':<15} {'0.33 bits/s/Hz':<20} {'Frequency shifts'}")
print(f"{'BPSK':<15} {f'{2*R_b} Hz':<15} {'0.5 bits/s/Hz':<20} {'Phase shifts, best SNR'}")
print("═" * 70)
print("\nBPSK Advantages:")
print("  • Same bandwidth as ASK, but 3 dB better noise performance")
print("  • 33% more bandwidth-efficient than FSK")
print("  • Constant envelope (power efficient)")
print("  • Foundation for QPSK, 8-PSK, and higher-order modulations")

## Part 5: BPSK Modulator Design

A BPSK modulator consists of:
1. **NRZ Encoder**: Converts bits (0,1) to bipolar levels (±A)
2. **Carrier Oscillator**: Generates $\cos(2\pi f_c t)$
3. **Product Modulator (Mixer)**: Multiplies NRZ and carrier

```
Bits → [NRZ Encoder] → [×] → BPSK Output
       (+1 or -1)       ↑
                  [cos(2πfct)]
```

In [ ]:
# BPSK Modulator Block Diagram Simulation

def nrz_encoder(bits, amplitude=1):
    """Convert bits to NRZ: 0→-A, 1→+A"""
    return amplitude * (2 * bits - 1)

def carrier_generator(t, f_c, amplitude=1):
    """Generate carrier signal"""
    return amplitude * np.cos(2 * np.pi * f_c * t)

def product_modulator(signal1, signal2):
    """Multiply two signals (mixer)"""
    return signal1 * signal2

# Test the modulator
test_bits = np.array([1, 0, 1, 1, 0, 1, 0, 0])
print(f"Test bit sequence: {test_bits}")

# Step 1: NRZ Encoding
nrz_levels = nrz_encoder(test_bits, amplitude=A)
print(f"NRZ levels: {nrz_levels}")

# Step 2: Create time-domain NRZ signal
t_test = np.arange(0, len(test_bits) * T_b, 1/fs)
nrz_signal = np.zeros(len(t_test))
for n in range(len(test_bits)):
    start_idx = int(n * T_b * fs)
    end_idx = int((n + 1) * T_b * fs)
    if end_idx <= len(nrz_signal):
        nrz_signal[start_idx:end_idx] = nrz_levels[n]

# Step 3: Generate carrier
carrier_signal = carrier_generator(t_test, f_c, amplitude=1)

# Step 4: Product modulation
bpsk_output = product_modulator(nrz_signal, carrier_signal)

# Visualize modulator blocks
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

# Input bits
axes[0, 0].step(np.arange(len(test_bits)), test_bits, where='post', linewidth=2, color='blue')
axes[0, 0].set_ylabel('Bits')
axes[0, 0].set_title('Input: Digital Bits')
axes[0, 0].set_ylim(-0.5, 1.5)
axes[0, 0].grid(True, alpha=0.3)

# NRZ levels (bar chart)
axes[0, 1].bar(np.arange(len(test_bits)), nrz_levels, color=['green' if b==1 else 'red' for b in test_bits])
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].set_title('Block 1: NRZ Encoder Output')
axes[0, 1].axhline(y=0, color='k', linewidth=0.5)
axes[0, 1].grid(True, alpha=0.3)

# NRZ signal (time domain)
axes[1, 0].plot(t_test * 1000, nrz_signal, linewidth=2, color='green')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].set_title('NRZ Signal (Time Domain)')
axes[1, 0].axhline(y=0, color='k', linewidth=0.5)
axes[1, 0].grid(True, alpha=0.3)

# Carrier signal
plot_len = int(3 * T_b * fs)
axes[1, 1].plot(t_test[:plot_len] * 1000, carrier_signal[:plot_len], linewidth=1, color='red')
axes[1, 1].set_ylabel('Amplitude')
axes[1, 1].set_title(f'Block 2: Carrier Generator ({f_c} Hz)')
axes[1, 1].axhline(y=0, color='k', linewidth=0.5)
axes[1, 1].grid(True, alpha=0.3)

# BPSK output (time domain)
axes[2, 0].plot(t_test[:plot_len] * 1000, bpsk_output[:plot_len], linewidth=1, color='purple')
axes[2, 0].plot(t_test[:plot_len] * 1000, nrz_signal[:plot_len], '--', linewidth=2, 
                color='green', alpha=0.5, label='Envelope')
axes[2, 0].set_xlabel('Time (ms)')
axes[2, 0].set_ylabel('Amplitude')
axes[2, 0].set_title('Block 3: Product Modulator Output (BPSK)')
axes[2, 0].legend()
axes[2, 0].axhline(y=0, color='k', linewidth=0.5)
axes[2, 0].grid(True, alpha=0.3)

# BPSK spectrum
S_test = fftshift(fft(bpsk_output))
f_test = fftshift(fftfreq(len(bpsk_output), 1/fs))
axes[2, 1].plot(f_test / 1000, np.abs(S_test), linewidth=1, color='purple')
axes[2, 1].axvline(x=f_c/1000, color='blue', linestyle='--', linewidth=2, label='Carrier')
axes[2, 1].set_xlabel('Frequency (kHz)')
axes[2, 1].set_ylabel('Magnitude')
axes[2, 1].set_title('BPSK Output Spectrum')
axes[2, 1].set_xlim(-15, 15)
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nBPSK Modulator Summary:")
print("═" * 70)
print("Block 1: NRZ Encoder")
print(f"  Input: Binary bits {test_bits}")
print(f"  Output: Bipolar levels {nrz_levels}")
print(f"  Mapping: Bit 0 → {-A}, Bit 1 → {+A}")
print("\nBlock 2: Carrier Generator")
print(f"  Frequency: {f_c} Hz")
print(f"  Waveform: cos(2π·{f_c}·t)")
print("\nBlock 3: Product Modulator (Mixer)")
print(f"  Operation: NRZ × Carrier")
print(f"  Output: BPSK signal with 180° phase shifts")
print("═" * 70)

## Summary and Key Takeaways

### BPSK Fundamentals
- **Binary Phase Shift Keying**: 180° phase difference between symbols
- **Antipodal signaling**: Maximum Euclidean distance for binary modulation
- **Constant envelope**: $|s(t)| = A$ (power efficient)

### Constellation
- One-dimensional: Two points at $(±A, 0)$ on I-axis
- Decision boundary at $I = 0$
- Energy per bit: $E_b = A^2 T_b / 2$

### Spectrum
- **Null-to-null BW**: $2R_b$
- **99% power BW**: $\approx 1.2 R_b$
- **Spectral efficiency**: $0.5$ bits/s/Hz

### Advantages
1. Best noise performance among binary modulations (3 dB better than ASK)
2. Same bandwidth as ASK
3. More bandwidth-efficient than FSK (33% improvement)
4. Foundation for QPSK, 8-PSK, and QAM

### Implementation
- Simple product modulator: NRZ × Carrier
- Coherent demodulation required (carrier recovery)
- Used in satellite, deep space, and military communications

### Next Topics
- BPSK demodulation and matched filters (Lecture 14)
- QPSK: 4-ary PSK (2 bits/symbol)
- M-ary PSK and QAM for higher data rates